In [1]:
import transformers.models.clip.image_processing_clip

In [2]:
!pip install langchain
!pip install qdrant-client
!pip install sentence_transformers
!pip install rank-bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.69.0
    Uninstalling grpcio-1.69.0:
      Successfully uninstalled grpcio-1.69.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but y

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [4]:
!pip install kaggle



In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small
!kaggle datasets download -d faresdyab/fashion-product-images-dataset



## **Import libraries**

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document
from langchain.vectorstores import Qdrant
import transformers.models.clip.image_processing_clip

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, HnswConfigDiff,OptimizersConfigDiff,ScalarQuantization,ScalarType, ScalarQuantizationConfig

import pandas as pd
from PIL import Image
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient

## **Load dataset**

In [ ]:
import pandas as pd
metadata_f = pd.read_csv("/kaggle/input/fashion-product-images-small/styles.csv", on_bad_lines='skip')
imagedata_f = pd.read_csv('/kaggle/input/fashion-product-images-dataset/fashion-dataset/images.csv',on_bad_lines='skip')
my_fashion = pd.read_csv('/kaggle/input/my-fashion/images.csv',on_bad_lines='skip')

In [ ]:
import pandas as pd

# Load metadata for the small dataset
metadata_f = pd.read_csv('./data/small_dataset/styles.csv', on_bad_lines='skip')

# Display basic information
print(metadata_f.head())


In [ ]:
import os

# Collect all image filenames for the small dataset
image_dir = './data/small_dataset/images'
image_filenames = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Create a DataFrame for image data
imagedata_f = pd.DataFrame({'filename': image_filenames})

# Display the first few rows
print(imagedata_f.head())


In [ ]:
import zipfile

# Extract the first dataset
with zipfile.ZipFile('fashion-product-images-small.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/small_dataset')

# Extract the second dataset
with zipfile.ZipFile('fashion-product-images-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/large_dataset')


In [ ]:
# Assuming imagedata_f and metadata_f are DataFrames containing relevant data
# combined_data = pd.concat([imagedata_f, metadata_f], axis=1)

# Filter rows where the 'subCategory' column is 'Topwear'
# combined_data = combined_data[combined_data['subCategory'] == 'Topwear']

# Select only the 'filename' and 'link' columns
# new_images = combined_data[['filename', 'link']]

# Assuming combined_data is your DataFrame
# combined_data['filename'] = combined_data['filename'].str.replace('.jpg', '').astype(float)

# Display the updated DataFrame
# combined_data

# imagedata_f['filename'] = imagedata_f['filename'].str.replace('.jpg', '').astype(float)

# combined_data = pd.merge(imagedata_f, metadata_f, left_on='filename', right_on='id', how='inner')

# # Drop rows with null values
# combined_data = combined_data.dropna()

# Display the combined and filtered DataFrame


In [ ]:
import pandas as pd

# Load metadata from the CSV file
metadata_f = pd.read_csv('./data/small_dataset/styles.csv')  # Update the path if necessary

In [ ]:
metadata_f = metadata_f[metadata_f['subCategory'] == 'Topwear']
# new_images = combined_data[['filename', 'link']]
metadata_f

## **Process dataset**

In [ ]:
SAMPLE_COUNT = 15402

In [ ]:
BASE_PATH = "/kaggle/input/fashion-product-images-small/images"
count = 0
metdata_rows = []
sampled_image_paths = []
for index, row in metadata_f.iterrows():

    if count == SAMPLE_COUNT:
        break

    path = f"{BASE_PATH}/{row.id}.jpg"
    if not os.path.exists(path):
        continue

    sampled_image_paths.append(Image.open(path))
    metdata_rows.append(row)
    count = count+1

sampled_metadata_df = pd.DataFrame(metdata_rows, columns = metadata_f.columns)

In [ ]:
sampled_metadata_df.drop(["year"], axis=1, inplace=True)

In [ ]:
def metadata_as_sentences():
    sentences = []

    for _, row in sampled_metadata_df.iterrows():
        # Join the values of the row except the first value (id)
        joined_values = " ".join([str(i) for i in row.values[1:]])

        # Append a tuple containing (id, joined_values) to the sentences list
        sentences.append((row['id'], joined_values))

    return sentences


In [ ]:
metadata_as_sentence = metadata_as_sentences()

In [ ]:
metadata_as_sentence[0:10]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def display_result(image_paths, metadata_sentences):
    """
    Displays images along with their metadata.

    Args:
        image_paths (list): List of paths to the sampled images.
        metadata_sentences (list): List of metadata descriptions corresponding to the images.
    """
    plt.figure(figsize=(15, 10))
    for i, (img_path, metadata) in enumerate(zip(image_paths, metadata_sentences)):
        # Load and display the image
        img = mpimg.imread(img_path)
        plt.subplot(2, 5, i + 1)  # Adjust rows and columns as needed
        plt.imshow(img)
        plt.axis('off')
        plt.title(metadata, fontsize=8)
    plt.tight_layout()
    plt.show()


In [ ]:
# Example call to display_result
display_result(sampled_image_paths[0:10], metadata_as_sentence[0:10])


In [ ]:
display_result(sampled_image_paths[0:10],metadata_as_sentence[0:10])

## Encode data

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load a CLIP model from huggingface
model = SentenceTransformer(
    'sentence-transformers/clip-ViT-B-32',
    device=device
)
model

#### Encode sparse vector

In [ ]:
sentence_as_documents = [Document(page_content=value, metadata={"image_id": _id}) for _id, value in metadata_as_sentence]
bm25_retriever = BM25Retriever.from_documents(sentence_as_documents)

In [ ]:
image_dense_embeddings = []

for i in tqdm(sampled_image_paths):
    image_dense_embeddings.append(model.encode(i))

#### Encode dense vector

In [ ]:
print(len(image_dense_embeddings), image_dense_embeddings[0].shape)

### **Persist in Vector database**

In [ ]:
user_secrets = UserSecretsClient()
vector_db_key = user_secrets.get_secret("vector_db_key")

In [ ]:
client = QdrantClient(
    url="https://763bc1da-0673-4535-91ac-b5538ec0287f.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=vector_db_key,
) # Persists changes to disk, fast prototyping

In [ ]:
COLLECTION_NAME="semantic_image_search"

In [ ]:
client.delete_collection(COLLECTION_NAME)

In [ ]:
client.create_collection(COLLECTION_NAME, vectors_config=VectorParams(size=512,  distance=Distance.COSINE))

In [ ]:
vector_line_batch = []
for i in tqdm(range(len(sampled_image_paths))):


    vector_line_batch.append(
        PointStruct(
                id=i,
                vector=image_dense_embeddings[i].tolist(),
                payload={"metadata":{"image_id": metadata_as_sentence[i][0]}, "page_content":metadata_as_sentence[i][1]}
            )
    )

    if i % 100 == 0:

        client.upsert(
                collection_name=COLLECTION_NAME,
                wait=False,
                points=vector_line_batch
            )
        vector_line_batch = []

client.upsert(
        collection_name=COLLECTION_NAME,
        wait=False,
        points=vector_line_batch
    )

### **Data Retrievers**

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name='sentence-transformers/clip-ViT-B-32')

In [ ]:
dense_vector_retriever = Qdrant(client, COLLECTION_NAME, embeddings)

In [ ]:
sparse_heavy_retiever = EnsembleRetriever(retrievers=[bm25_retriever, dense_vector_retriever.as_retriever()], weights=[0.8, 0.2])
dense_heavy_retiever = EnsembleRetriever(retrievers=[bm25_retriever, dense_vector_retriever.as_retriever()], weights=[0.2, 0.8])
neutral_retiever = EnsembleRetriever(retrievers=[bm25_retriever, dense_vector_retriever.as_retriever()], weights=[0.5, 0.5])

In [ ]:
def search_and_show(retirever, query):
    search_result1 = retirever.get_relevant_documents(query)
    filtered_images, captions = get_results(search_result1)
    return display_result(filtered_images, captions)

In [ ]:
SEARCH_QUERY = "sun colour wear"
search_and_show(neutral_retiever, SEARCH_QUERY)

In [ ]:
search_and_show(sparse_heavy_retiever, SEARCH_QUERY)

In [ ]:
search_and_show(dense_heavy_retiever, SEARCH_QUERY)

In [ ]:
SEARCH_QUERY2 = "Blue clothes for men"

search_result2 = dense_vector_retriever.as_retriever().get_relevant_documents(SEARCH_QUERY2)
filtered_images2 = get_results(search_result2)
# display_result(filtered_images2, captions2)
print(filtered_images2)

In [ ]:
my_fashion.loc[2832]
metadata_f.loc[2832]
sampled_metadata_df.loc[2832]

## Util methods

In [ ]:
from IPython.core.display import HTML
from io import BytesIO
from base64 import b64encode

# function to display product images
def display_result(image_batch, caption_batch):
    figures = []
    for i in range(len(caption_batch)):

        img = image_batch[i]
        caption = caption_batch[i]

        b = BytesIO()
        img.save(b, format='png')
        figures.append(f'''
            <figure style="margin: 5px !important;">

              <img src="data:image/png;base64,{b64encode(b.getvalue()).decode('utf-8')}" style="width: 90px; height: 120px" ></br>
              <span style="display:block; width:150px; word-wrap:break-word;"> {caption} </span>
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

In [ ]:
def get_results(search_results):
    filtered_img_ids = [doc.metadata.get("image_id") for doc in search_results]
#     filtered_images = [sampled_image_paths[_id] for _id in filtered_img_ids]
    print(filtered_img_ids)

#     captions = [doc.page_content for doc in search_results]
    return filtered_img_ids